# Computing persistent homologies

## Simplicial complex

In [1]:
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
#warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

from sklearn.preprocessing import quantile_transform

%matplotlib inline

In [3]:
full = pd.read_csv('data/CA1_6_1_clean.csv')

In [4]:
mis = pd.read_csv('data/mi_best_neurons.csv', index_col=0);

In [5]:
#data = df.drop(['x', 'y', 'segment'], axis=1)[mi_best_neurons.index]

In [6]:
from dionysus import Filtration, homology_persistence, init_diagrams, Simplex
from collections import Counter
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def get_cycles_for_filtration(filtration, end_time=np.inf):
    m = homology_persistence(filtration, prime=2)
    dgms = init_diagrams(m, filtration)
    cycles = []
    for i, dgm in enumerate(dgms):
        for pt in dgm:
            cycles.append((i, pt.birth, pt.death))
            #print((i, pt.birth,pt.death))
    cycles = pd.DataFrame(cycles, columns=['dimension', 'birth_time', 'death_time'])
    cycles['living_time'] = cycles['death_time'] - cycles['birth_time']
    inf_life = (cycles['living_time'] == np.inf)
    cycles['living_time'][inf_life] = end_time - cycles['birth_time'][inf_life]
    return cycles.sort_values('living_time', ascending=False)



def homologies(df, number_of_ap):
    # time - simplexes that live > time%
    # number_of_ap = (time/100) * len(df)
    #number_of_ap=90
    #print(number_of_ap)

    df = quantile_transform(df)
    time = 10
    filtration = Filtration()
    for alpha in np.arange(0.50, 0.99, 0.01):
        #print("alpha", alpha)
        neuron_activity = data > alpha
        
        complexes = []
        sims = np.sum(neuron_activity * (2**np.arange(neuron_activity.shape[1])), axis=1)
        #print(Counter(sims))
        for sim, cnt in Counter(sims).items():
            if cnt > number_of_ap:
                complexes.append(np.where(np.array(list(bin(int(sim))[2:][::-1])) == '1')[0])
        
        all_simplexes = []
        for compl in complexes:
            for s in list(powerset(compl))[1:]:
                all_simplexes.append(s)
                
                
        all_simplexes = sorted(list(set(all_simplexes)), key=lambda x: len(x))
        for sim in all_simplexes:
            filtration.append(Simplex(sim, alpha))
        #print(filtration)
        #print()
    cycles = get_cycles_for_filtration(filtration, 0.95)
    #cycles.columns = [f"{col}_{number_of_ap/df.shape[0]}" for col in cycles.columns]
    
    return cycles.reset_index(drop=True)

In [7]:
for n in [2, 3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500][:]:
    print(n)
    mi_best_neurons = mis[:10]['neuron']
    data = full.drop(['x', 'y', 'segment'], axis=1)[mi_best_neurons.index]
    df = homologies(data, n)
    print(df.T)
    print()

2
                0
dimension    0.00
birth_time   0.50
death_time    inf
living_time  0.45

3
                0     1
dimension    0.00  4.00
birth_time   0.50  0.50
death_time    inf   inf
living_time  0.45  0.45

5
                0
dimension    0.00
birth_time   0.50
death_time    inf
living_time  0.45

10
                0     1     2
dimension    0.00  3.00  3.00
birth_time   0.50  0.50  0.50
death_time    inf   inf  0.51
living_time  0.45  0.45  0.01

20
                0     1     2     3     4
dimension    0.00  3.00  3.00  3.00  3.00
birth_time   0.50  0.50  0.50  0.50  0.50
death_time    inf   inf   inf   inf   inf
living_time  0.45  0.45  0.45  0.45  0.45

30
                0     1     2     3
dimension    0.00  3.00  3.00  3.00
birth_time   0.50  0.50  0.50  0.50
death_time    inf   inf   inf   inf
living_time  0.45  0.45  0.45  0.45

40
                0     1     2     3
dimension    0.00  2.00  2.00  2.00
birth_time   0.50  0.50  0.50  0.51
death_time    inf   inf   in

In [9]:
for n in [0, 1, 2, 3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500][8:12]:
    print(n)
    mi_best_neurons = mis[:24]['neuron']
    data = full.drop(['x', 'y', 'segment'], axis=1)[mi_best_neurons.index]
    df = homologies(data, n)
    print(df.T)
    print()

40
                0     1     2
dimension    0.00  1.00  1.00
birth_time   0.50  0.82  0.91
death_time    inf   inf   inf
living_time  0.45  0.13  0.04

50
                0     1     2     3
dimension    0.00  1.00  1.00  0.00
birth_time   0.62  0.84  0.90  0.93
death_time    inf   inf   inf   inf
living_time  0.33  0.11  0.05  0.02

60
                0     1     2     3     4     5     6
dimension    0.00  0.00  0.00  0.00  0.00  0.00  0.00
birth_time   0.63  0.76  0.84  0.80  0.92  0.93  0.94
death_time    inf  0.86  0.90  0.85   inf   inf   inf
living_time  0.32  0.10  0.06  0.05  0.03  0.02  0.01

70
                0     1     2     3     4     5     6     7     8     9  \
dimension    0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00   
birth_time   0.79  0.81  0.85  0.90  0.86  0.93  0.93  0.93  0.94  0.86   
death_time    inf   inf   inf   inf  0.91   inf   inf   inf   inf  0.87   
living_time  0.16  0.14  0.10  0.05  0.05  0.02  0.02  0.02  0.01  0.01   

         

# Playground (not related at all)

In [48]:
alpha=0.60

In [62]:
neuron_activity = data > alpha

In [63]:
neuron_activity = neuron_activity

In [64]:
sims = np.sum(neuron_activity * 2**np.arange(neuron_activity.shape[1]), axis=1)

In [65]:
c = Counter(sims)

In [66]:
common_cnt=20 # at least 2 codes for simplex exist

In [67]:
# get complexes (3 of which > common_cnt)
complexes = []
for sim, cnt in c.items():
    print(sim, cnt)
    print(bin(int(sim)))
    print(bin(int(sim))[2:]) # remove 0b
    print(bin(int(sim))[2:][::-1]) # reversed
    print(list(bin(int(sim))[2:][::-1]))
    print(np.array(list(bin(int(sim))[2:][::-1])) == '1')
    print(np.where(np.array(list(bin(int(sim))[2:][::-1])) == '1'))
    if cnt > common_cnt:
        complexes.append(np.where(np.array(list(reversed(bin(int(sim))[2:]))) == '1')[0])
    print()

2148 9
0b100001100100
100001100100
001001100001
['0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1']
[False False  True False False  True  True False False False False  True]
(array([ 2,  5,  6, 11]),)

2404 3
0b100101100100
100101100100
001001101001
['0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '1']
[False False  True False False  True  True False  True False False  True]
(array([ 2,  5,  6,  8, 11]),)

3428 1
0b110101100100
110101100100
001001101011
['0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1']
[False False  True False False  True  True False  True False  True  True]
(array([ 2,  5,  6,  8, 10, 11]),)

2340 2
0b100100100100
100100100100
001001001001
['0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '0', '1']
[False False  True False False  True False False  True False False  True]
(array([ 2,  5,  8, 11]),)

2084 20
0b100000100100
100000100100
001001000001
['0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1']
[False False  True False False  Tr

(array([ 3,  6,  9, 11, 12]),)

23112 3
0b101101001001000
101101001001000
000100100101101
['0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '1']
[False False False  True False False  True False False  True False  True
  True False  True]
(array([ 3,  6,  9, 11, 12, 14]),)

39496 11
0b1001101001001000
1001101001001000
0001001001011001
['0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1']
[False False False  True False False  True False False  True False  True
  True False False  True]
(array([ 3,  6,  9, 11, 12, 15]),)

39488 1
0b1001101001000000
1001101001000000
0000001001011001
['0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1']
[False False False False False False  True False False  True False  True
  True False False  True]
(array([ 6,  9, 11, 12, 15]),)

39624 2
0b1001101011001000
1001101011001000
0001001101011001
['0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1']
[False False

['0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0', '1']
[False False False  True False False  True False False False  True False
  True]
(array([ 3,  6, 10, 12]),)

6218 34
0b1100001001010
1100001001010
0101001000011
['0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1']
[False  True False  True False False  True False False False False  True
  True]
(array([ 1,  3,  6, 11, 12]),)

4186 1
0b1000001011010
1000001011010
0101101000001
['0', '1', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '1']
[False  True False  True  True False  True False False False False False
  True]
(array([ 1,  3,  4,  6, 12]),)

4160 2
0b1000001000000
1000001000000
0000001000001
['0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1']
[False False False False False False  True False False False False False
  True]
(array([ 6, 12]),)

4672 4
0b1001001000000
1001001000000
0000001001001
['0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1']
[False False False False Fals

(array([ 2,  3, 10, 11, 13]),)

10252 4
0b10100000001100
10100000001100
00110000000101
['0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1']
[False False  True  True False False False False False False False  True
 False  True]
(array([ 2,  3, 11, 13]),)

33132 1
0b1000000101101100
1000000101101100
0011011010000001
['0', '0', '1', '1', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1']
[False False  True  True False  True  True False  True False False False
 False False False  True]
(array([ 2,  3,  5,  6,  8, 15]),)

33588 2
0b1000001100110100
1000001100110100
0010110011000001
['0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '0', '1']
[False False  True False  True  True False False  True  True False False
 False False False  True]
(array([ 2,  4,  5,  8,  9, 15]),)

32884 1
0b1000000001110100
1000000001110100
0010111000000001
['0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1']
[False False  True False  True 

0b100011010110001
100011010110001
100011010110001
['1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1']
[ True False False False  True  True False  True False  True  True False
 False False  True]
(array([ 0,  4,  5,  7,  9, 10, 14]),)

26289 1
0b110011010110001
110011010110001
100011010110011
['1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']
[ True False False False  True  True False  True False  True  True False
 False  True  True]
(array([ 0,  4,  5,  7,  9, 10, 13, 14]),)

26291 11
0b110011010110011
110011010110011
110011010110011
['1', '1', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1']
[ True  True False False  True  True False  True False  True  True False
 False  True  True]
(array([ 0,  1,  4,  5,  7,  9, 10, 13, 14]),)

17585 7
0b100010010110001
100010010110001
100011010010001
['1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1']
[ True False False False  True  True False  True False False

(array([ 1,  2,  3, 11, 15]),)

14 3
0b1110
1110
0111
['0', '1', '1', '1']
[False  True  True  True]
(array([1, 2, 3]),)

32780 1
0b1000000000001100
1000000000001100
0011000000000001
['0', '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1']
[False False  True  True False False False False False False False False
 False False False  True]
(array([ 2,  3, 15]),)

34894 3
0b1000100001001110
1000100001001110
0111001000010001
['0', '1', '1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1']
[False  True  True  True False False  True False False False False  True
 False False False  True]
(array([ 1,  2,  3,  6, 11, 15]),)

2062 1
0b100000001110
100000001110
011100000001
['0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1']
[False  True  True  True False False False False False False False  True]
(array([ 1,  2,  3, 11]),)

43022 31
0b1010100000001110
1010100000001110
0111000000010101
['0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0

(array([ 0,  4,  5,  8, 11]),)

22929 1
0b101100110010001
101100110010001
100010011001101
['1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '1']
[ True False False False  True False False  True  True False False  True
  True False  True]
(array([ 0,  4,  7,  8, 11, 12, 14]),)

23953 3
0b101110110010001
101110110010001
100010011011101
['1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '1', '1', '0', '1']
[ True False False False  True False False  True  True False  True  True
  True False  True]
(array([ 0,  4,  7,  8, 10, 11, 12, 14]),)

23985 1
0b101110110110001
101110110110001
100011011011101
['1', '0', '0', '0', '1', '1', '0', '1', '1', '0', '1', '1', '1', '0', '1']
[ True False False False  True  True False  True  True False  True  True
  True False  True]
(array([ 0,  4,  5,  7,  8, 10, 11, 12, 14]),)

22417 2
0b101011110010001
101011110010001
100010011110101
['1', '0', '0', '0', '1', '0', '0', '1', '1', '1', '1', '0', '1', '0', '1']
[ True False False Fa

In [68]:
# get all simplexes
all_simplexes = []
for compl in complexes:
    for s in list(powerset(compl))[1:]:
        all_simplexes.append(s)

In [69]:
len(all_simplexes)

9233

In [82]:
all_simplexes[0]

(3,)

In [60]:
filtration = Filtration()

In [74]:
set(all_simplexes)

{(0, 1, 4, 8),
 (1, 2, 8),
 (1, 4, 5, 7, 8, 10, 14),
 (1, 4, 8, 10, 14, 15),
 (3, 5, 12, 15),
 (3, 6, 11),
 (0, 1, 4, 8, 9, 10, 15),
 (0, 1, 7, 8, 10, 13, 14, 15),
 (0, 2, 4, 5, 7, 9),
 (7, 8, 10, 14, 15),
 (4, 5, 8, 9, 13),
 (2, 3, 5, 11),
 (1, 4, 8, 13),
 (3, 5, 6, 11, 12, 15),
 (0, 4, 10),
 (7, 9, 14),
 (1, 4, 8, 10, 13),
 (1, 3, 13, 15),
 (2, 5, 8, 13),
 (5, 8, 15),
 (1, 4, 8, 13, 15),
 (0, 4, 9, 14),
 (4, 7, 14),
 (5, 11, 13, 15),
 (2, 4, 7, 8, 10, 14),
 (3, 5, 11, 15),
 (1, 9, 14, 15),
 (0, 4, 7, 14, 15),
 (2, 4, 5, 7, 8),
 (0, 9, 10, 13),
 (2, 6, 15),
 (5, 7, 9, 13),
 (2, 4, 5, 7),
 (1, 8, 9, 13, 14, 15),
 (1, 5, 7, 8, 9, 10, 14),
 (2, 3, 6, 11, 12, 13, 15),
 (1, 2, 4, 7, 8, 9, 14),
 (0, 14),
 (0, 7, 8, 10, 13, 14),
 (0, 2, 5, 7, 9, 10, 14),
 (3, 11),
 (7, 14, 15),
 (0, 1, 4, 8, 14, 15),
 (0, 1, 4, 7, 8, 9, 10, 14, 15),
 (0, 4, 7, 8, 9, 10, 13, 15),
 (2, 4, 5, 8, 11, 12),
 (0, 2, 5, 7, 8, 9, 14),
 (7, 15),
 (4, 6, 13),
 (1, 4, 7, 8, 9, 10, 15),
 (6, 9, 13, 15),
 (0, 1, 2, 4, 7, 

In [61]:
all_simplexes = sorted(list(set(all_simplexes)), key=lambda x: len(x))

In [72]:
for sim in all_simplexes:
    filtration.append(Simplex(sim, alpha))

In [73]:
filtration

Filtration with 2636 simplices

#  Simple example

In [10]:
alpha=0.5

In [11]:
simplices = [
    [1, 2, 9, 11],
    [9, 10, 11],
    [7,8,10],
    [6,7,8],
    [3,4,5],
    [2,3],
    [3,6],
    [7,9]
]

In [38]:
simplices = [
    #[1, 2, 3],
    [1, 2],
    [2, 3],
    [1, 3],
    [2, 4],
    [4, 3],
]

In [39]:
all_simplexes = []
for compl in simplices:
    for s in list(powerset(compl))[1:]:
        all_simplexes.append(s)
all_simplexes = sorted(list(set(all_simplexes)), key=lambda x: len(x))
filtration = Filtration()
for sim in all_simplexes:
    filtration.append(Simplex(sim, 1 - alpha))

In [40]:
filtration

Filtration with 9 simplices

In [41]:
f = filtration

In [42]:
m = homology_persistence(filtration)

In [43]:
dgms = init_diagrams(m, f)
for i, dgm in enumerate(dgms):
    print("Dimension:", i)
    for p in dgm:
        print(p)

Dimension: 0
(0.5,inf)
Dimension: 1
(0.5,inf)
(0.5,inf)


In [19]:
dgms = init_diagrams(m, f)
for i, dgm in enumerate(dgms):
    print("Dimension:", i)
    for p in dgm:
        print(p)

Dimension: 0
(0.5,inf)
Dimension: 1
(0.5,inf)
(0.5,inf)
Dimension: 2
Dimension: 3
